<a href="https://colab.research.google.com/github/itamarbeeri/gittry/blob/master/build_error2size_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
data_dir='/content/drive/My Drive/project with Roi Livni/data'
output_size = 10
epochs = 200
batch_size = 100
learning_rate = 0.1
momentum = 0.5
train_size=50000
test_size=10000

dataset_names = ['MNIST'] #, 'CIFAR10']
hidden_layer_sizes = [8, 128]
my_transforms = transforms.Compose([transforms.Resize((10, 10)), ToTensor()])


In [15]:
print('mount drive..')
from google.colab import drive
drive.mount('/content/drive/')


mount drive..
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


input dot weights of same size
relu
dot hidden layer of some size
output size
turcated softmax
total params = H + input size

cross entropy loss
SGD with momentum and diminishing step size
minibatches 100
init step size 0.1 with update rule
init momentum 0.5 with update rule


downsample to 100 pixels
50000 MNIST 40000 CIFAR
10000 validation

increasing H will reduse training and test error initially and then overfit - increase test error.


In [16]:
print('imports and initialization..')

from os import path, mkdir

import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset
from torchvision import datasets
from torchvision.transforms import ToTensor, transforms
from torchsummary import summary
from functools import reduce
import uuid
import yaml
from pprint import pprint
from torch.optim.lr_scheduler import MultiplicativeLR


imports and initialization..


In [17]:
print('mount GPU..')

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.empty_cache()
else:
    device = torch.device("cpu")
    print("No Cuda Available")


print(f"Using {device} device")

mount GPU..
Using cuda:0 device


In [19]:
class LNN(nn.Module):
    def __init__(self, input_shape, hidden_layer_size, output_size=10):
        super().__init__()
        self.flatten = nn.Flatten()
        input_layer_size = reduce((lambda x, y: x * y), input_shape)
        self.architecture = nn.Sequential(
                        nn.Linear(input_layer_size, hidden_layer_size),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size, output_size),
                        nn.Softmax())

    def forward(self, x):
        x = self.flatten(x)
        logits = self.architecture(x)
        return logits

In [20]:
def build_model(input_shape, hidden_layer_size, output_size, learning_rate, momentum):
    model = LNN(input_shape, hidden_layer_size, output_size).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
    scheduler = MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.99)

    return model, loss_fn, optimizer, scheduler


def build_data(dataset_name='MNIST', my_transforms=None, batch_size=32, train_size=60000, test_size=10000):
    Data = list()
    dataset = datasets.CIFAR10 if dataset_name == 'CIFAR10' else datasets.MNIST

    for is_train, size in zip([True, False], [train_size, test_size]):
        data = dataset(root="data", train=is_train, download=True, transform=my_transforms)
        used_data, _ = torch.utils.data.random_split(data, [size, len(data) - size])
        dataloader = DataLoader(used_data, batch_size=batch_size, shuffle=True, num_workers=8)
        Data.append((used_data, dataloader))

    return Data


In [21]:

def save_history(training_history, data_dir, experiment_dict):
    dest_dir = path.join(data_dir, experiment_dict['exp_name'])
    print(f'saving history to {dest_dir}..')

    if path.isdir(dest_dir):
        print(f'dir {dest_dir} already exist!')
    else:
        mkdir(dest_dir)

    loss = np.asarray(training_history['loss_history'])
    np.save(path.join(dest_dir, f'loss.npy'), loss)

    test_loss = np.asarray(training_history['test_loss_history'])
    np.save(path.join(dest_dir, f'test_loss.npy'), test_loss)

    layers = list(training_history['gradients_history'][0].keys())
    for layer in layers:
        weights = list()
        for batch in training_history['weights_history']:
            weights.append(np.asarray(batch[layer].cpu()))
        np.save(path.join(dest_dir, f'weights_{layer}.npy'), weights)

        gradients = list()
        for batch in training_history['gradients_history']:
            gradients.append(np.asarray(batch[layer].cpu()))
        np.save(path.join(dest_dir, f'gradients_{layer}.npy'), gradients)

        test_gradients = list()
        for batch in training_history['test_gradients_history']:
            test_gradients.append(np.asarray(batch[layer].cpu()))
        np.save(path.join(dest_dir, f'test_gradients_{layer}.npy'), test_gradients)

    with open(path.join(data_dir, 'experiments.yml'),'r') as file:
        experiments = yaml.safe_load(file)
        exp_name = experiment_dict.pop('exp_name')
        experiments[exp_name] = experiment_dict

    with open(path.join(data_dir, 'experiments.yml'), 'w') as file:
        yaml.safe_dump(experiments, file)

    print(f'data saved in {dest_dir}.')

In [22]:
def train(epoch):
    model.train()
    for i, (images, labels) in enumerate(train_dataloader):
        images, labels = images.to(device), labels.to(device)

        output = model(images)
        loss = loss_fn(output, labels)

        if i % 10 == 0:
            print('Epoch %d, Batch: %d, Loss: %f' % (epoch, i, loss.detach().cpu().item()))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return loss.detach().cpu().item()


def test():
    model.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(test_dataloader):
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        avg_loss += loss_fn(output, labels).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(test_set)
    avg_loss *= batch_size
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(test_set)))

    return avg_loss.detach().cpu().item()


In [23]:

for dataset_name in dataset_names:
    (train_set, train_dataloader), (test_set, test_dataloader) = build_data(dataset_name, my_transforms, batch_size=batch_size, train_size=train_size, test_size=test_size)
    batch_sample, label = next(iter(train_dataloader))
    sample = np.asarray(batch_sample[0,:,:,:])

    for hidden_layer_size in hidden_layer_sizes:
        model, loss_fn, optimizer, scheduler = build_model(sample.shape, hidden_layer_size, output_size, learning_rate, momentum)

        experiment_dict = dict()
        experiment_dict['exp_name'] = str(uuid.uuid4())
        experiment_dict['learning_rate'] = learning_rate
        experiment_dict['epochs'] = epochs
        experiment_dict['batch_size'] = batch_size
        experiment_dict['train_set_size'] = len(train_set)
        experiment_dict['test_set_size'] = len(test_set)
        experiment_dict['sample_shape'] = sample.shape
        experiment_dict['dataset_name'] = dataset_name
        experiment_dict['transforms'] = str(my_transforms)
        experiment_dict['model_summary'] = str(model)
        experiment_dict['loss'] = 'cross_entropy'
        pprint(experiment_dict)

        print('run training..')
        prev_test_loss = 0
        for epoch in range(epochs):
            train_loss = train(epoch)
            test_loss = test()

            scheduler.step()
            if np.abs(prev_test_loss - test_loss) < 0.000001:
              print('constant test loss!')
              break
            prev_test_loss = test_loss

        # save_history(history, data_dir=data_dir, experiment_dict=experiment_dict)
        experiment_dict['hidden_layer_size'] = hidden_layer_size
        experiment_dict['final_train_loss'] = train_loss
        experiment_dict['final_test_loss'] = test_loss
        experiment_dict['convergence_epoch'] = epoch

        with open(path.join(data_dir, 'hidden_layer_loss.yml'),'r') as file:
          experiments = yaml.safe_load(file)
          exp_name = experiment_dict.pop('exp_name')
          experiments[exp_name] = experiment_dict

        with open(path.join(data_dir, 'hidden_layer_loss.yml'), 'w') as file:
          yaml.safe_dump(experiments, file)
        print('file saved')

print('done')


{'batch_size': 100,
 'dataset_name': 'MNIST',
 'epochs': 200,
 'exp_name': 'cec73f0e-59da-44e4-b5d4-1b0d7325894b',
 'learning_rate': 0.1,
 'loss': 'cross_entropy',
 'model_summary': 'LNN(\n'
                  '  (flatten): Flatten(start_dim=1, end_dim=-1)\n'
                  '  (architecture): Sequential(\n'
                  '    (0): Linear(in_features=100, out_features=8, '
                  'bias=True)\n'
                  '    (1): ReLU()\n'
                  '    (2): Linear(in_features=8, out_features=10, bias=True)\n'
                  '    (3): Softmax(dim=None)\n'
                  '  )\n'
                  ')',
 'sample_shape': (1, 10, 10),
 'test_set_size': 10000,
 'train_set_size': 50000,
 'transforms': 'Compose(\n'
               '    Resize(size=(10, 10), interpolation=bilinear, '
               'max_size=None, antialias=warn)\n'
               '    ToTensor()\n'
               ')'}
run training..


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch 0, Batch: 0, Loss: 2.303377
Epoch 0, Batch: 10, Loss: 2.300911
Epoch 0, Batch: 20, Loss: 2.302065
Epoch 0, Batch: 30, Loss: 2.301735
Epoch 0, Batch: 40, Loss: 2.298720
Epoch 0, Batch: 50, Loss: 2.299842
Epoch 0, Batch: 60, Loss: 2.301872
Epoch 0, Batch: 70, Loss: 2.295636
Epoch 0, Batch: 80, Loss: 2.302032
Epoch 0, Batch: 90, Loss: 2.300038
Epoch 0, Batch: 100, Loss: 2.300833
Epoch 0, Batch: 110, Loss: 2.304341
Epoch 0, Batch: 120, Loss: 2.296189
Epoch 0, Batch: 130, Loss: 2.299995
Epoch 0, Batch: 140, Loss: 2.296740
Epoch 0, Batch: 150, Loss: 2.295904
Epoch 0, Batch: 160, Loss: 2.289294
Epoch 0, Batch: 170, Loss: 2.291524
Epoch 0, Batch: 180, Loss: 2.291625
Epoch 0, Batch: 190, Loss: 2.285177
Epoch 0, Batch: 200, Loss: 2.291354
Epoch 0, Batch: 210, Loss: 2.283694
Epoch 0, Batch: 220, Loss: 2.281820
Epoch 0, Batch: 230, Loss: 2.293262
Epoch 0, Batch: 240, Loss: 2.299969
Epoch 0, Batch: 250, Loss: 2.288693
Epoch 0, Batch: 260, Loss: 2.288265
Epoch 0, Batch: 270, Loss: 2.278044
Epo

KeyboardInterrupt: ignored